# Converting 3D Data to Projection Data and Cross Correlating

In order to test the convolutional neural network with the use of cross correlation the training and testing data will be the projections instead of the stack of projections.

In [1]:
# Import essential packages
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import tifffile
from tifffile import imsave

# Import tomography and imaging packages
import tomopy
from skimage.transform import rotate, AffineTransform
from skimage import transform as tf
from scipy.fft import fft2, fftshift
from scipy.signal import correlate

# Import neural net packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.profiler
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torchinfo import summary

In [2]:
# Checking to ensure environment and cuda are correct
print("Working Environment: {}".format(os.environ['CONDA_DEFAULT_ENV']))
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print("Cuda Version: {}".format(torch.version.cuda))
print("Cuda Availability: {}".format(torch.cuda.is_available()))

Working Environment: pytorch
Cuda Version: 11.8
Cuda Availability: True


In [3]:
# Loading data, 25 entries of 128 resolution shepp3ds
res = 128
entries = 25
data = []

for i in range(entries):
    data.append(np.load('./shepp{}-{}/shepp{}-{}_{}.npy'.format(res, entries, res, entries, i), 
                        allow_pickle = True))
    
data = np.asarray(data)

In [20]:
projections = np.zeros((entries * data[0][0][0, 0].shape[0], 2), dtype = object)

for i in range (entries):
    
    for j in range (data[0][0][0, 0].shape[0]):
        
        projections[i * data[0][0][0, 0].shape[0] + j, 0] = data[i, 0][0, 0, j, :, :]
        projections[i * data[0][0][0, 0].shape[0] + j, 1] = np.asarray([data[i, 1][0, 2 * j], 
                                                                        data[i, 1][0, 2 * j + 1]])

In [21]:
print(projections.shape)
print(projections[0, 0].shape)
print(projections[0, 1].shape)

(4500, 2)
(128, 184)
(2,)


In [22]:
def to_projections(data, entries):
    
    projections = np.zeros((entries * data[0][0][0, 0].shape[0], 2), dtype = object)

    for i in range (entries):

        for j in range (data[0][0][0, 0].shape[0]):

            projections[i * data[0][0][0, 0].shape[0] + j, 0] = data[i, 0][0, 0, j, :, :]
            projections[i * data[0][0][0, 0].shape[0] + j, 1] = np.asarray([data[i, 1][0, 2 * j], 
                                                                            data[i, 1][0, 2 * j + 1]])
            
    return projections

In [25]:
proj_data = to_projections(data, entries)

print(proj_data.shape)
print(proj_data[0, 0].shape)
print(proj_data[0, 1].shape)

(4500, 2)
(128, 184)
(2,)


In [26]:
print(data.shape)
print(data[0, 0][0, 0].shape)
print(data[0, 1].shape)

(25, 2)
(180, 128, 184)
(1, 360)


Now that we can see that splitting the data into projections work, now we can perform the operation on cross correlation data to be fed into the neural network.

In [27]:
def crosscorr_reprojection(data, entries):
    
    ang = tomopy.angles(data[0][0][0, 0].shape[0])
    _rec = 1e-12 * np.ones((data[0][0][0, 0].shape[1], data[0][0][0, 0].shape[2], data[0][0][0, 0].shape[2]))
    data_copy = data.copy()
    out_data = np.zeros((entries, 2), dtype = object)
    
    for i in range (entries):
    
        out_data[i, 0] = np.zeros((1, 1, data[0][0][0, 0].shape[0], data[0][0][0, 0].shape[1] * 2 - 1,
                                      data[0][0][0, 0].shape[2] * 2 - 1))
        out_data[i, 1] = data_copy[i, 1]
        
        rec = tomopy.recon(data_copy[i][0][0, 0], ang, center = None, 
                            algorithm = 'mlem', init_recon = _rec)
        reproj = tomopy.project(rec, ang, center = None, pad = False)
        
        for j in range (data[0][0][0, 0].shape[0]):
            
            out_data[i, 0][0, 0, j] = correlate(data_copy[i][0][0, 0, j], reproj[j], method = 'fft')
        
    return out_data

In [28]:
crosscorr_reproj = crosscorr_reprojection(data.copy(), entries)

print(crosscorr_reproj.shape)
print(crosscorr_reproj[0, 0].shape)
print(crosscorr_reproj[0, 1].shape)

(25, 2)
(1, 1, 180, 255, 367)
(1, 360)


In [29]:
crosscorr_data = to_projections(crosscorr_reproj, entries)

print(crosscorr_data.shape)
print(crosscorr_data[0, 0].shape)
print(crosscorr_data[0, 1].shape)

(4500, 2)
(255, 367)
(2,)


As we can see, the data for training and testing the neural network can now be properly set up.